In [24]:
import requests
import json
from pathlib import Path

In [2]:
url = "http://localhost:11434/api/chat"
model = "llama3.1"

In [17]:
def query_model(prompt, model="llama3.1", url="http://localhost:11434/api/chat"):
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "options": {
            "seed": 123,
            "temperature": 0,
        }
    }

    response = requests.post(url, json=data)
    response_data = ""
    # decode response content
    for line in response.content.decode("utf-8").split("\n"):
        if line:
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]
    return response_data

In [22]:
resp = query_model("TonTon Friend")

In [25]:

json_file = Path("instruct_data.json")

with open(json_file, "r") as file:
    json_data = json.load(file)

print("Number of entries:", len(json_data))

Number of entries: 40


In [26]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. Write a response that "
        f"appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    instruction_text + input_text

    return instruction_text + input_text

In [27]:
import random


for entry in json_data[:5]:
    
    politeness = random.choice(["polite", "impolite"])    
    prompt = (
        f"Given the input `{format_input(entry)}` "
        f"and correct output `{entry['output']}`, "
        f"slightly rewrite the output to be more {politeness}."
        "Keep the modification minimal."
        "Only return return the generated response and nothing else."
    )
    print("\nDataset response:")
    print(">>", entry['output'])
    print(f"\n{politeness} response:")
    print(">>", query_model(prompt)) 


Dataset response:
>> The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".

polite response:
>> The spelling of the given phrase "freind" is not quite right, the correct spelling is actually "friend".

Dataset response:
>> He goes to the park every day.

impolite response:
>> He goes to the park every single day, okay?

Dataset response:
>> 45 kilometers is 45000 meters.

polite response:
>> You are correct, 45 kilometers is equivalent to 45,000 meters.

Dataset response:
>> Although it was raining, they went for a walk.

impolite response:
>> Although it was bloody well raining, they still decided to go for a walk.

Dataset response:
>> 1, 4, 9, 16, 25, 36, 49, 64, 81, 100.

impolite response:
>> Here are the first 10 square numbers: 1, 4, 9, 16, 25, 36, 49, 64, 81, 100. Don't bother memorizing them, you'll just forget anyway.
